In [1]:
%load_ext autoreload
%autoreload 2

from owlready2 import *
import owlready2
print(owlready2.VERSION)

import pandas as pd
import glob, os, hashlib
import requests, json
from datetime import datetime

import random

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

from dotenv import load_dotenv
load_dotenv('.env')

URL = os.getenv("KG_URL_FCT")
TOK3N = os.getenv("KG_TOKEN")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


0.40


In [2]:
import importlib.util
import sys

spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

In [3]:
onto = get_ontology("./WIP.owl").load()

# Techs


In [4]:
TECHS = [x.label[0] for x in onto.Technology.instances() if len(x.label) >= 1]
len(TECHS)

23090

In [5]:
def classify_technology(CategoriesOfTechs):
    return [
        {
        "name": "classify_technology",
        "description": "Function used to classify technologies from a high level perspective.",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_technology": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of technology ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The technology being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":CategoriesOfTechs,
                                "description": 'The category of technology that matches the technology being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct technology category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_technology"],
        },
    }
]

In [6]:
DICT = {}
DICT["Air Control"] = ['Air Filtering',
 'Indoor Environment',
 'HVAC Systems',
 'Air Quality Monitoring',
 'Air Purification',
 'Ventilation',
 'HEPA Filters',
 'UV-C Disinfection']
DICT["Data Analytics"] = ['Machine Learning and AI Algorithms', 'Real-Time Data Tracking and Monitoring', 'Data Visualization and Modeling', 'Cloud-Based Data Sharing and Analysis', 'Advanced Statistical Modeling and Analysis', 'Social Media Monitoring and Analysis', 'Predictive Analytics and Modeling Tools',  'Information Dissemination Platforms']
DICT["Infrastructure Management"] = ['Sanitation and hygiene facilities', 'Ventilation and air quality systems', 'Geographic information and mapping technology', 'Healthcare infrastructure and telecommunication networks', 'Energy-efficient technologies and smart grid systems', 'Remote work tools and workforce management technologies', 'Logistics and inventory management systems', 'Capacity-building and infrastructure development tools', 'Disaster management and resilience planning technologies']
DICT["Communication"] = ['Communication and Collaboration Platforms', 'Public Awareness Campaign Tools', 'Video Conferencing and Collaboration Software', 'Social Media Platforms', 'Virtual Communication and Collaboration Technologies', 'Digital Signage and Messaging Systems', 'Multilingual Communication and Outreach Tools', 'Online Public Engagement Platforms', 'Remote Work and Telecommuting Technologies', 'Mobile App Notification Systems']
DICT["Sensors and Monitoring"] = ['Monitoring solutions', 'IoT devices', 'Surveillance systems', 'Environmental sensors', 'Sensor technologies', 'Remote monitoring tools', 'Crowd monitoring technology', 'Data collection systems', 'Real-time tracking systems']
DICT["Healthcare"] = ['Diagnostic Testing', 'Sequencing and Genomic Analysis', 'Analytical Chemistry', 'Physical Health Monitoring', 'Wellbeing and Mental Health', 'Social Distancing Technologies', 'Vaccination Development and Distribution', 'Personal Hygiene and Sanitation', 'Personal Protective Equipment (PPE)', 'Remote Healthcare and Telemedicine']
DICT["Urban Planning"] = ['Community Management Technologies', 'Spatial Planning and Layout Tools', 'Green Infrastructure Management', 'Crowd Control Solutions', 'Smart City Technologies', 'Urban Planning Solutions', 'Resilience Planning Frameworks', 'Urban Design and Mapping Tools', 'Environmental Conservation Technologies']
DICT["Digital and Information Management"] = ['Data Management', 'Web Development', 'Network Security', 'Cloud Services', 'Mobile Applications', 'Knowledge Management', 'Ontology Tools', 'Data Governance', 'Digital Infrastructure', 'Information Systems']
DICT["Building design"] = ['Building design', 'Ventilation systems', 'Home design and architecture', 'Sustainable design', 'Soundproofing technology', 'Green building strategies', 'Space optimization technologies', 'Architectural design software', 'Adaptive construction methods', 'Virtual planning and design tools']
DICT["Privacy Enhancing Technologies"] = ['Encryption and authentication', 'Biometrics and identification', 'Face and voice recognition', 'Data encryption and security', 'Cybersecurity and access control', 'Quantum cryptography', 'Ethical guidelines and advocacy', 'Fact-checking and information verification', 'Blockchain for data privacy and transparency']
DICT["Building Management"] = ['Access control and security technology', 'Lighting and daylighting systems', 'Maintenance and landscaping equipment', 'Door-locking and automatic systems', 'Operation of HVAC and ventilation systems', 'Building management and automation systems', 'Industry standard compliance tools', 'Safety and hazard detection systems']
DICT["Cleaning"] = ['Automated sanitation equipment', 'Advanced disinfection systems', 'Surface cleaning', 'Waste management technologies group', 'Sanitization equipment group', 'Disinfectant formulation', 'Self-cleaning technologies', 'Hygiene maintenance tools']
DICT["Supply-chain"] = ['Diversification and suppliers', 'Supply chain and risk management',  'Manufacturing of PPE', 'Digital supply chain management', 'Contactless delivery']
DICT["Water Control"] = ['Water and Wastewater Management', 'Water Treatment Systems', 'Water Quality Monitoring', 'Water Disinfection Solutions',  'Advanced Filtration Technologies', 'Sewage Treatment Facilities', 'Water Recycling and Conservation']
DICT["Waste Management"] = ['Recycling and waste handling technologies', 'Advanced waste sorting and processing systems', 'Smart waste handling and monitoring technologies', 'Contactless recycling and waste disposal solutions', 'Automated waste collection and segregation technologies', 'Sanitization and sterilization technologies for waste', 'Improved waste storage and disposal systems']

CategoriesOfTechs = list(DICT.keys())

#### Doing Level 1 of technologies

In [7]:
RES = []
NChunks = 50
#Randomizing list
if os.path.isfile("../../support/newtechs_classified_lv1.parquet.gzip"):
    GOOD = pd.read_parquet("../../support/newtechs_classified_lv1.parquet.gzip")
    alreadythere = GOOD["name"].unique()
    newRisks = [x for x in TECHS if x not in alreadythere]
    CATS_FWK = [GOOD]
else:
    newRisks = TECHS
    CATS_FWK = []

if len(newRisks):
    newRisks = [str(x) for x in newRisks]
    NB = len(newRisks)//NChunks

    for k in range(NB):
        try:
            QUESTION =  "The technologies are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
            prefix ="technologies_"
            overwrite = False
            REQ = {
                "context": "Out the following list, classify the technologies from a high level perspective.\n\n",
                "question": QUESTION,
                "functions": classify_technology(CategoriesOfTechs),
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalTechsT3.5",
                "seed" : ""
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_technology"])
            d = d [d["category"].isin(CategoriesOfTechs)]
            CATS_FWK.append(d)
            DFC = pd.concat(CATS_FWK).reset_index(drop=True)
            DFC.to_parquet("../../support/newtechs_classified_lv1.parquet.gzip",compression="gzip")
            print("Done:",k)
        except:
            print("error")
            pass

Done: 0
Done: 1
Done: 2
Done: 3
Done: 4
Done: 5
Done: 6
Done: 7
Done: 8
Done: 9
Done: 10
Done: 11
Done: 12
Done: 13
Done: 14
Done: 15
Done: 16
Done: 17


#### Doing Level 2 of technologies

In [ ]:
DFC = pd.read_parquet("../../support/newtechs_classified_lv1.parquet.gzip")

DFC.head(2)

,name,category,confidence,co```nfidence,&confidence,subcategory,confidence_lv2
0,Nan,Privacy Enhancing Technologies,Low,None,None,Encryption and authentication,Low
1,Floor markings,Urban Planning,Medium-high,None,None,Spatial Planning and Layout Tools,High


In [ ]:
DFC = pd.read_parquet("../../support/newtechs_classified_lv1.parquet.gzip")
CATs = list(DFC.category.unique())
if "subcategory" not in DFC.columns:
    DFC["subcategory"] = None
if "confidence_lv2" not in DFC.columns:
    DFC["confidence_lv2"] = None
for CAT in CATs:
    print("Doing",CAT)
    df = DFC[DFC.category == CAT]
    DONE = list(df[~df.subcategory.isna()].name.unique())
    df = df[~df.name.isin(DONE)]
    subtechs = DICT[CAT] # classify_technology(CategoriesOfTechs)

    newRisks = list(df.name)
    RES = []
    NChunks = 50

    if len(newRisks):
        newRisks = [str(x) for x in newRisks]
        NB = len(newRisks)//NChunks

        for k in range(NB)[:2]:
            try:
                QUESTION =  "The technologies are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
                prefix ="technologies_"
                overwrite = False
                REQ = {
                    "context": "Out the following list, classify the technologies from a high level perspective.\n\n",
                    "question": QUESTION,
                    "functions": classify_technology(subtechs),
                    "token": TOK3N,
                    "overwrite": overwrite,
                    "source": "local-evalTechsT3.5",
                    "seed" : ""
                }

                H = h.hashme(REQ["context"]+REQ["question"])
                cached = "cache/"+prefix+"_"+H+".json"
                if not os.path.isfile(cached) or overwrite:
                    x = requests.post(URL+"fct/", json = REQ)
                    answer = json.loads(x.text)["messages"][-1]
                    h.svt(cached,answer)
                else:
                    answer = h.ldt(cached)
                d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_technology"])
                d = d [d["category"].isin(subtechs)]
                d.columns = ["name","subcategory","confidence_lv2"]
                for ix, row in d.iterrows():
                    DFC.loc[DFC.name == row["name"], "subcategory"] = row["subcategory"]
                    DFC.loc[DFC.name == row["name"], "confidence_lv2"] = row["confidence_lv2"]
                DFC.to_parquet("../../support/newtechs_classified_lv1.parquet.gzip",compression="gzip")
                print("Done:",k)
            except:
                print("error")
                pass

Doing Privacy Enhancing Technologies
Done: 0
Done: 1
Doing Urban Planning
Done: 0
Done: 1
Doing Healthcare
Done: 0
Done: 1
Doing Communication
Done: 0
Done: 1
Doing Building design
Done: 0
Done: 1
Doing Water Control
Done: 0
Done: 1
Doing Data Analytics
Done: 0
Done: 1
Doing Infrastructure Management
Done: 0
Done: 1
Doing Sensors and Monitoring
Done: 0
Done: 1
Doing Building Management
Done: 0
Done: 1
Doing Digital and Information Management
Done: 0
Done: 1
Doing Cleaning
Done: 0
Done: 1
Doing Air Control
Done: 0
Done: 1
Doing Waste Management
Done: 0
Done: 1
Doing Supply-chain
Done: 0
Done: 1


In [ ]:
import pandas as pd
DFC = pd.read_parquet("../../support/newtechs_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["subcategory"]).name.count()

27224 items listed


subcategory
Access control and security technology        12
Adaptive construction methods                  7
Advanced Filtration Technologies              16
Advanced Statistical Modeling and Analysis    38
Advanced disinfection systems                 39
                                              ..
Water Recycling and Conservation              13
Water Treatment Systems                       37
Water and Wastewater Management               74
Web Development                               26
Wellbeing and Mental Health                   23
Name: name, Length: 126, dtype: int64

# Stakeholders

In [ ]:
SH = [x.label[0] for x in onto.Stakeholder.instances() if len(x.label) >= 1]
len(SH)

16001

In [ ]:
DICT = {'Health Organisations': ['Public health authorities',
  'Occupational health and safety teams',
  'Government health agencies ',
  'International health organizations',
  'Medical and healthcare personnel',
  'Private health organisations',
  'Mental healthcare providers',
  'Veterinary organization'],
 'Research Organisations': ['AI and Data analysis researchers',
  'Research institutions ',
  'Technology developers',
  'Academic institutions',
  'Data repository administrators',
  'Local researchers',
  'Citizen researchers',
  'Scientific communities'],
 'Government institutions': ['City authorities',
  'Government departments ',
  'Policymakers',
  'Diplomatic bodies',
  'Local government',
  'Educational organizations',
  'Political communities',
  'Administrative authorities',
  'Regulatory bodies ',
  'Emergency services and management',
  'International development agencies'],
 'General public': ['Immigrant population',
  'Patients',
  'Travelers',
  'Civil society stakeholders',
  'Recreational and fitness enthusiasts',
  'Visitors in indoor spaces',
  'General population ',
  'Digitally connected individuals',
  'Fragile and/or isolated population',
  'Individual with high health risks',
  'Green spaces users '],
 'Environmental and sustainability organizations': ['Environmental agencies',
  'Conservation organizations',
  'Park management stakeholders',
  'Waste management organizations',
  'Sustainability teams ',
  'Energy and environmental consultants',
  'Green area managers and organizations',
  'Wildlife and ecosystem conservationists',
  'Climate and sustainability research institutions'],
 'Residents and occupants of spaces': ['Homeowners ',
  'Building managers',
  'People with lower income',
  'People in high-contamination areas',
  'Occupants of shared household spaces',
  'Visitors to green spaces',
  'Individuals working from home'],
 'Workers': ['Office workers',
  'Physical workers',
  'Healthcare workers',
  'Real estate agents',
  'Facility maintenance team',
  'AI system developers',
  'Waste management workers',
  'Project managers'],
 'Regulatory bodies': ['Global governing bodies',
  'Trade regulators',
  'Enforcement organizations',
  'Certification system organizations',
  'Standardization bodies',
  'Data governance organizations',
  'Governments and industry regulatory bodies',
  'Security agency',
  'Occupational health and safety regulators',
  'Local regulatory authorities',
  'International trade regulators'],
 'Public Entities': ['Sports-related organizations',
  'Appointed personnel responsible for prevention measures',
  'Education-related entities',
  'Water utilities',
  'Energy Utilities',
  'Tourism organizations ',
  'Culture organizations',
  'Police ',
  'Public communication authorities'],
 'Financial groups': ['Banking institutions',
  'Funding Bodies',
  'Economic analysts and regulatory authorities',
  'International financial institutions',
  'Treasury departments',
  'Small-and medium-sized enterprises',
  'Companies in general',
  'Global markets',
  'Insurances providers',
  'Finance professionals',
  'Social entrepreneurship fund managers',
  'Investors',
  'Social entrepreneurship funds'],
 'Construction and infrastructure industry': ['Building design team',
  'Workers in the construction industry',
  'Construction industry organizations',
  'Contractors ',
  'HVAC equipment providers',
  'Site-supervision workers',
  'Operational staff',
  'Design and construction professionals',
  'Building maintenance and green building professionals',
  'Facility maintenance staff'],
 'Manufacturing and production sector': ['Production staff',
  'Non-medical cechnology manufacturers',
  'Medical technology manufacturers',
  'Food industry companies',
  'International suppliers',
  'Supply chain professionals',
  'Wearable technology industry',
  'Pharmaceutical industry'],
 'Transportation and mobility sector': ['Logistics professionals',
  'Ride-sharing ',
  'City transport authorities',
  'Vehicle users',
  'Safe mobility',
  'Soft mobility',
  'Route optimization specialists',
  'Airports',
  'Ports',
  'Railways',
  'Delivery services',
  'Logistics and transportation managers'],
 'Cities professionals': ['City planning professionals',
  'Urban development authorities',
  'Individual urban planners and designers',
  'Urban planning institutions',
  'Smart city professionals',
  'Sustainable city professionals',
  'International urban institutions',
  'Public space managers'],
 'Retail and service industry': ['Cleaning personnel',
  'Retail staff',
  'Marketing agency',
  'Local businesses',
  'Food distribution',
  'Animal rearing industry',
  'Online retailers',
  'Drive-through staff',
  'Supermarket and health-care facility staff',
  'Parking lot owners',
  'Local fitness businesses'],
 'Energy Sector': ['Energy monitoring system providers',
  'Operators and maintenance staff ',
  'Renewable energy investors',
  'Energy management authorities',
  'Energy communities',
  'Communities reliant on the coal and oil industries',
  'Global energy markets',
  'Local energy markets',
  'Energy grid operators'],
 'Media organizations': ['Local communication experts',
  'Individuals consuming digital content',
  'Media and news entities',
  'Marketing/advertising agencies',
  'Reporter',
  'Social media platform providers',
  'Academic journals',
  'News outlet'],
 'Cities Officials': ['Municipality officials',
  'Public building managers',
  'City authorities ',
  'Public space managers',
  'Local government and urban development authorities',
  'Community leaders',
  'Smart city managers',
  'Other cities departments']}

In [ ]:
def classify_SH(CategoriesOfStakeholders):
    return [
        {
        "name": "classify_SH",
        "description": "Function used to classify stakeholders from a high level perspective, based on what group they are most closely belonging to. ",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_stakeholders": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of stakeholder ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The stakeholder being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":CategoriesOfStakeholders,
                                "description": 'The category of risk that matches the stakeholder category being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct stakeholder category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_stakeholders"],
        },
    }
]

In [ ]:
CategoriesOfStakeholders = DICT.keys()
RES = []
NChunks = 50
#Randomizing list
if os.path.isfile("../../support/newstakeholders_classified_lv1.parquet.gzip"):
    GOOD = pd.read_parquet("../../support/newstakeholders_classified_lv1.parquet.gzip")
    alreadythere = GOOD["name"].unique()
    newRisks = [x for x in SH if x not in alreadythere]
    CATS_FWK = [GOOD]
else:
    newRisks = SH
    CATS_FWK = []
if len(newRisks):
    NB = len(newRisks)//NChunks
    for k in range(NB):
        try:
            QUESTION =  "The stakeholders are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
            prefix ="sh_"
            overwrite = False
            REQ = {
                "context": "Out the following list, classify the stakeholders from a high level perspective.\n\n",
                "question": QUESTION,
                "functions": classify_SH(list(CategoriesOfStakeholders)),
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalSHT3.5",
                "seed" : ""
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_stakeholders"])
            d = d [d["category"].isin(CategoriesOfStakeholders)]
            CATS_FWK.append(d)
            DFC = pd.concat(CATS_FWK).reset_index(drop=True)
            DFC.to_parquet("../../support/newstakeholders_classified_lv1.parquet.gzip",compression="gzip")
            print("Done:",k)
        except:
            print("error")
            pass
else:
    print("All risks covered")

Done: 0
Done: 1
Done: 2
Done: 3
Done: 4
Done: 5
Done: 6
Done: 7
Done: 8
Done: 9
Done: 10
Done: 11
Done: 12
Done: 13
Done: 14
Done: 15
Done: 16
Done: 17
Done: 18
Done: 19
Done: 20
Done: 21
Done: 22
Done: 23
Done: 24
Done: 25
Done: 26
Done: 27
Done: 28
Done: 29
Done: 30
Done: 31
Done: 32
Done: 33
Done: 34
Done: 35
Done: 36
Done: 37
Done: 38
Done: 39
Done: 40
Done: 41
Done: 42
Done: 43
Done: 44
Done: 45
Done: 46
Done: 47
Done: 48
Done: 49
Done: 50
Done: 51
Done: 52
Done: 53
Done: 54
Done: 55
Done: 56
Done: 57
Done: 58
Done: 59
Done: 60
error
Done: 62
Done: 63
Done: 64
Done: 65
Done: 66
Done: 67
Done: 68
Done: 69
Done: 70
Done: 71
Done: 72
Done: 73
Done: 74
Done: 75
Done: 76
Done: 77
Done: 78
Done: 79
Done: 80
Done: 81
Done: 82
Done: 83
Done: 84
Done: 85
Done: 86
Done: 87
Done: 88
Done: 89
Done: 90
Done: 91
Done: 92
Done: 93
Done: 94
Done: 95
Done: 96
Done: 97
Done: 98
Done: 99
Done: 100
Done: 101
Done: 102
Done: 103
Done: 104
Done: 105
Done: 106
Done: 107
Done: 108
Done: 109
Done: 110
Don

In [ ]:
import pandas as pd
DFC = pd.read_parquet("../../support/newstakeholders_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["category"]).name.count()

10875 items listed


category
Cities Officials                                    70
Cities professionals                                67
Construction and infrastructure industry           496
Energy Sector                                      114
Environmental and sustainability organizations     420
Financial groups                                   219
General public                                    1028
Government institutions                            820
Health Organizations                              1573
Manufacturing and production sector                333
Media organizations                                159
Private Sector                                    1175
Public Entities                                    439
Regulatory bodies                                  162
Research Organisations                             801
Residents and occupants of spaces                 2455
Retail and service industry                        298
Transportation and mobility sector                 246
N

In [ ]:
DFC = pd.read_parquet("../../support/newstakeholders_classified_lv1.parquet.gzip")
CATs = list(DFC.category.unique())
if "subcategory" not in DFC.columns:
    DFC["subcategory"] = None
if "confidence_lv2" not in DFC.columns:
    DFC["confidence_lv2"] = None
for CAT in CATs:
    print("Doing",CAT)
    df = DFC[DFC.category == CAT]
    DONE = list(df[~df.subcategory.isna()].name.unique())
    df = df[~df.name.isin(DONE)]
    subtechs = DICT[CAT] # classify_technology(CategoriesOfTechs)

    newRisks = list(df.name)
    RES = []
    NChunks = 50

    if len(newRisks):
        newRisks = [str(x) for x in newRisks]
        NB = len(newRisks)//NChunks

        for k in range(NB)[:2]:
            try:
                QUESTION =  "The stakeholders are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
                prefix ="technologies_"
                overwrite = False
                REQ = {
                    "context": "Out the following list, classify the stakeholders from a high level perspective.\n\n",
                    "question": QUESTION,
                    "functions": classify_SH(subtechs),
                    "token": TOK3N,
                    "overwrite": overwrite,
                    "source": "local-evalTechsT3.5",
                    "seed" : ""
                }

                H = h.hashme(REQ["context"]+REQ["question"])
                cached = "cache/"+prefix+"_"+H+".json"
                if not os.path.isfile(cached) or overwrite:
                    x = requests.post(URL+"fct/", json = REQ)
                    answer = json.loads(x.text)["messages"][-1]
                    h.svt(cached,answer)
                else:
                    answer = h.ldt(cached)
                d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_technology"])
                d = d [d["category"].isin(subtechs)]
                d.columns = ["name","subcategory","confidence_lv2"]
                for ix, row in d.iterrows():
                    DFC.loc[DFC.name == row["name"], "subcategory"] = row["subcategory"]
                    DFC.loc[DFC.name == row["name"], "confidence_lv2"] = row["confidence_lv2"]
                DFC.to_parquet("../../support/newstakeholders_classified_lv1.parquet.gzip",compression="gzip")
                print("Done:",k)
            except:
                print("error")
                pass

# Risks


In [ ]:
RISKS = [x.label[0] for x in onto.Risk.instances() if len(x.label) >= 1]
len(RISKS)

20897

In [ ]:
DICT = {'Political risks': ['Trust and transparency challenges',
  'Globalization challenges',
  'Lack of global governance and coordination',
  'Weak local decision-making mechanisms',
  'Lack of coordination and consistency in mitigation measures',
  'Ineffectiveness of interventions',
  'Bioterrorism threats',
  'Global coordination challenges',
  'Lack of transparency and clarity'],
 'Social risks': ['Disinformation and Misinformation',
  'Social and Psychological Effects ',
  'Social Justice Concerns',
  'Communication and Messaging',
  'Access to infrastructure  ',
  'Physical activity and mobility',
  'Misinformation and Media Risks',
  'Cultural and Lifestyle Risks',
  'Vulnerability risks',
  'Occupational and Employment Risks',
  'Mental and Emotional Wellbeing',
  'Health Behaviors and Compliance'],
 'Infrastructure risks': ['Inadequate remote work setup for employees',
  'Improper architectural designs',
  'Inadequate mobility schemes',
  'Poor urban planning',
  'Infrastructure maintenance risk ',
  'Lack of emphasis on air quality ',
  'Lack of emphasis on water quality ',
  'Insufficient adoption of health-focused building standards',
  'Insufficient adoption of health-focused urban standards',
  'Inadequate hospital design ',
  'Inadequate adaptability to emerging problems  ',
  'Disruptions from outbreaks and other major external events'],
 'Public health risks': ['Failure to implement preventive measures',
  'Misinterpretation of data and results',
  'Lack of access to health services',
  'Public health messaging and education',
  'Lack of effective medical solutions',
  'Healthcare resource shortage and management',
  'Workplace safety and occupational exposure',
  'Poor access to Diagnostics and Treatment',
  'Poor vector-borne diseases control'],
 'Data Privacy and Security risks': ['Data Security risks',
  'Privacy risks',
  'Cybersecurity Threats',
  'Inaccurate or Incomplete Data',
  'Misuse of Personal Data',
  'Reliability and Quality of Data',
  'Surveillance and Monitoring Privacy Concerns',
  'Limited Accessibility and Data Equity',
  'Transparency and Accountability Issues',
  'Potential Misinformation and False Data'],
 'Economic risks': ['Supply chain disruptions',
  'Pandemic impact on markets',
  'Lower investment',
  'Lower employment',
  'Discontinuity of services',
  'Global recession and economic instability',
  'Construction industry impacts',
  'Resource and capability risks',
  'Workforce and labor shortages',
  'Financial strains and liquidity crises',
  'Food supply chain disruptions '],
 'Legal risks': ['Compliance and Contractual',
  'Data and Research Integrity',
  'Intellectual Property',
  'Workplace Safety and Health',
  'Privacy and Civil Liberties',
  'Fraud and Misconduct',
  'Healthcare Legislation',
  'Building Codes and Certification',
  'Technology Deployment and Surveillance'],
 'Environmental risks': ['Climate change',
  'Pollution increase',
  'Environmental health risks',
  'Air quality and pollution effects',
  'Urbanization and habitat loss',
  'Climate change effects',
  'Wastewater and sanitation risks',
  'Biodiversity and ecological impacts',
  'Energy consumption and sustainability issues'],
 'Technological risks': ['Data privacy and security risks',
  'Healthcare and medical risks',
  'Infrastructure and connectivity risks',
  'Modeling and simulation risks',
  'Modeling and prediction risks',
  'Artificial intelligence and machine learning risks',
  'Inadequate ventilation and air circulation',
  'Operational and implementation risks ',
  'Innovation and adoption risks',
  'Healthcare infrastructure and technology adoption risks ']}

In [ ]:
CategoriesOfRisks = list(DICT.keys())

In [ ]:
def classify_risks_PESTLE(CategoriesOfRisks):
    return [
        {
        "name": "classify_risks_PESTLE",
        "description": "Function used to classify risks from a high level perspective, based on what they  impact the most . ",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_risks": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of risk ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The risk being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":CategoriesOfRisks,
                                "description": 'The category of risk that matches the risk category being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct risk category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_risks"],
        },
    }
]

In [ ]:
RES = []
NChunks = 50
#Randomizing list
if os.path.isfile("../../support/newrisks_classified_lv1.parquet.gzip"):
    GOOD = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
    alreadythere = GOOD["name"].unique()

    newRisks = [x for x in RISKS if x not in alreadythere]
else:
    newRisks = RISKS
    
if len(newRisks):
    newRisks = [str(x) for x in newRisks]
    NB = len(newRisks)//NChunks
    CATS_FWK = [GOOD]
    for k in range(NB)[:150]:
        try:
            QUESTION =  "The risks are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
            prefix ="risks_"
            overwrite = False
            REQ = {
                "context": "Out the following list, classify the risks from a high level perspective.\n\n",
                "question": QUESTION,
                "functions": classify_risks_PESTLE(CategoriesOfRisks),
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalrisksT3.5",
                "seed" : ""
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_risks"])
            d = d [d["category"].isin(CategoriesOfRisks)]
            CATS_FWK.append(d)
            DFC = pd.concat(CATS_FWK).reset_index(drop=True)
            DFC.to_parquet("../../support/newrisks_classified_lv1.parquet.gzip",compression="gzip")
            print("Done:",k)
        except:
            print("error")
            pass
else:
    print("All risks covered")

error
error
error


In [ ]:
import pandas as pd
DFC = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["category"]).name.count()

3981


category
Data Privacy and Security risks     204
Economic risks                      397
Environmental risks                 404
Infrastructure risks                344
Legal risks                          36
Political risks                     161
Public health risks                1269
Social risks                        758
Technological risks                 408
Name: name, dtype: int64

In [ ]:
DFC = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
CATs = list(DFC.category.unique())
if "subcategory" not in DFC.columns:
    DFC["subcategory"] = None
if "confidence_lv2" not in DFC.columns:
    DFC["confidence_lv2"] = None
for CAT in CATs:
    print("Doing",CAT)
    df = DFC[DFC.category == CAT]
    DONE = list(df[~df.subcategory.isna()].name.unique())
    df = df[~df.name.isin(DONE)]
    subtechs = DICT[CAT] # classify_technology(CategoriesOfTechs)

    newRisks = list(df.name)
    RES = []
    NChunks = 50

    if len(newRisks):
        newRisks = [str(x) for x in newRisks]
        NB = len(newRisks)//NChunks

        for k in range(NB)[:2]:
            try:
                QUESTION =  "The risks are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
                prefix ="technologies_"
                overwrite = False
                REQ = {
                    "context": "Out the following list, classify the risks from a high level perspective.\n\n",
                    "question": QUESTION,
                    "functions": classify_risks_PESTLE(subtechs),
                    "token": TOK3N,
                    "overwrite": overwrite,
                    "source": "local-evalTechsT3.5",
                    "seed" : ""
                }

                H = h.hashme(REQ["context"]+REQ["question"])
                cached = "cache/"+prefix+"_"+H+".json"
                if not os.path.isfile(cached) or overwrite:
                    x = requests.post(URL+"fct/", json = REQ)
                    answer = json.loads(x.text)["messages"][-1]
                    h.svt(cached,answer)
                else:
                    answer = h.ldt(cached)
                d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_technology"])
                d = d [d["category"].isin(subtechs)]
                d.columns = ["name","subcategory","confidence_lv2"]
                for ix, row in d.iterrows():
                    DFC.loc[DFC.name == row["name"], "subcategory"] = row["subcategory"]
                    DFC.loc[DFC.name == row["name"], "confidence_lv2"] = row["confidence_lv2"]
                DFC.to_parquet("../../support/newrisks_classified_lv1.parquet.gzip",compression="gzip")
                print("Done:",k)
            except:
                print("error")
                pass

## Mitigs


In [ ]:
m = [x.label[0] for x in onto.Mitigation.instances() if len(x.label) >= 1]
len(m)

22884

In [ ]:
interventions = [x.label[0] for x in onto.BP_Intervention.instances()]

In [ ]:
def classify_mitigation(interventions):
    return [
        {
        "name": "classify_mitigation",
        "description": "Function used to classify mitigations from a high level perspective, based on what group they are most closely belonging to. ",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_mitigations": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of mitigation ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The mitigation being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":interventions,
                                "description": 'The category of mitigation that matches the mitigation category being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct mitigation category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_mitigations"],
        },
    }
]

In [ ]:
RES = []
NChunks = 50
#Randomizing list
if os.path.isfile("../../support/newmitigation_classified_lv1.parquet.gzip"):
    GOOD = pd.read_parquet("../../support/newmitigation_classified_lv1.parquet.gzip")
    alreadythere = GOOD["name"].unique()
    newRisks = [x for x in m if x not in alreadythere]
    CATS_FWK = [GOOD]
else:
    newRisks = m
    CATS_FWK = []
if len(newRisks):
    NB = len(newRisks)//NChunks
    for k in range(NB)[:150]:
        try:
            QUESTION =  "The mitigations are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
            prefix ="sh_"
            overwrite = False
            REQ = {
                "context": "Out the following list, classify the mitigations from a high level perspective.\n\n",
                "question": QUESTION,
                "functions": classify_mitigation(interventions),
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalMitigationsT3.5",
                "seed" : ""
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_mitigations"])
            d = d [d["category"].isin(interventions)]
            CATS_FWK.append(d)
            DFC = pd.concat(CATS_FWK).reset_index(drop=True)
            DFC.to_parquet("../../support/newmitigation_classified_lv1.parquet.gzip",compression="gzip")
            print("Done:",k)
        except:
            print("error")
            pass
else:
    print("All mitigations covered")

error
Done: 1
Done: 2
error
error
error
Done: 6
Done: 7
error
error
error
error
error
error
error
error
error
error
Done: 18
error
Done: 20
error
error
Done: 23
Done: 24
error
error
error
Done: 28
Done: 29
Done: 30
error
Done: 32
Done: 33
error
error
error
error
Done: 38
error
error
Done: 41
Done: 42
error
Done: 44
error
error
Done: 47
error
Done: 49
Done: 50
error
error
error
Done: 54
error
Done: 56
Done: 57
error
Done: 59
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
Done: 77
error
Done: 79
error
error
error
error
error
error
error
Done: 87
error
error
error
error
error
error
Done: 94
error
error
error
Done: 98
Done: 99
error
error
error
error
error
Done: 105
error
error
error
Done: 109
error
Done: 111
Done: 112
error
error
Done: 115
Done: 116
error
error
error
Done: 120


In [ ]:
import pandas as pd
DFC = pd.read_parquet("../../support/newmitigation_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["category"]).name.count()

2222 items listed


category
Absorb resource shortages                                             42
Absorb surge in service demande                                       78
Air filtration                                                        25
Animal vectors control                                                36
Cleaning technologies (robots)                                        14
Control airflows in spaces & Avoid air recirculation                  48
Control of airflows (airtight / negative pressure)                     8
Disinfection of air                                                   41
Early detection                                                       80
Ensure use of personal protection equipment                           61
Environmental parameters control                                     214
Facilitate transparent communication                                  61
Greenery                                                              40
Greywater management                      